# Description of policies used in this code
* The goal is to make a master DataFrame containing all info relevant to planning an observation session.
* The work of building this DataFrame is done by several functions, all with similar signature, that can be chained together.
* The pipeline() function takes a list of these functions, and pipes the output of one into the input of another.
* Each function adds/deletes/modifies one or more columns to the input DataFrame.
* NOTE: currently there is the side effect that the DataFrame input to a function is often modified and then returned by the function.
* All the columns added by a given function have names that start with a common prefix and then an underscore.
* Snake casing is used for column names so there is a one-to-one correspondence between column and attribute names.
* To ease prefix renaming, functions define their prefix in a variable that is then used wherever possible.
* The observing plan is made for a given location and between specified start and end times, usually all or part of a day.
* Some common portions of column & variable names used:
  * _beg, _end: start and finish of observing session
  * _jd: Julian Date
  * _time: local time at the observatory


In [53]:
import warnings
from sqlite3 import connect, Connection
import astropy as apy
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
import astropy.units as u
from astropy.utils.exceptions import AstropyWarning
import astroplan as ap
import pandas as pd


# Set Location and Observing session date/time here

In [54]:
# observer = ap.Observer.at_site("Gemini North", timezone="US/Hawaii")
# observer = ap.Observer.at_site("lbt", timezone="US/Arizona")
observer = ap.Observer.at_site("Apache Point Observatory", timezone="US/Mountain")

session_beg, session_end = observer.tonight(Time('2024-05-28', format='iso', scale='utc'), horizon=-6*u.deg)

#other initial setup stuff
jd_beg, jd_end = session_beg.jd, session_end.jd

conn = connect("../django/TargetDB/db.sqlite3")


In [55]:
# PEPSI exposure time calculator
import numpy as np
import math

def get_object_flux_dictionary():

    spectypes=['O5V', 'O9V', 'B0V', 'B1V', 'B3V', 'B6V', 'B8V', 'A0V', 'A2V', 'A3V', 'A5V', 'F0V', 'F2V']
    spec_str='F5V, F8V, G0V, G2V, G5V, G8V, K0V, K2V, K5V, K7V, M0V, M2V, M4V, M5V, B2IV, B6IV, A0IV, '
    spec_str=spec_str+'A4-7IV, F0-2IV, F5IV, F8IV, G0IV, G2IV, G5IV, G8IV, K0IV, K1IV, K3IV, O8III, B1-2III, B5III, '
    spec_str = spec_str+'B9III, A0III, A5III, F0III, F5III, G0III, G5III, G8III, K0III, K3III, K5III, M0III, M5III, '
    spec_str = spec_str+ 'M10III, B2II, B5II, F0II, F2II, G5II, K0-1II, K3-4II, M3II, B0I, B5I, B8I, A0I, F0I, '
    spec_str = spec_str+'F5I, F8I, G0I, G5I, G8I, K2I, K4I, M2I, Seyfert1, Seyfert2, QSO, NGC1068, Liner'

    spectypes = spectypes+spec_str.split(', ')



    flux_array=[[239593039.763, 178543473.813, 124031554.257, 82534178.9555, 52094588.3886, 30216798.7116],
                [224137694.103, 172205034.458, 126093669.89, 80045933.2903, 51523158.0048, 30126736.1341],
                [228955744.058, 174893335.89, 123225191.932, 83093894.2227, 53409615.9756, 31982647.2655],
                [203867603.322, 163612745.223, 121765478.951, 84705239.0309, 54203841.1893, 34916553.3623],
                [193000258.246, 159633040.67, 120739509.738, 83739958.6425, 56950404.9026, 37220541.4696],
                [196964790.184, 156444685.295, 119223858.616, 85247685.164, 60502251.1293, 40693294.6208],
                [185401043.818, 147695755.167, 113467540.659, 87920455.11, 60709980.127, 42093478.8237],
                [167909073.769, 146713180.85, 116465787.385, 89571584.9871, 64926699.746, 43691869.1895],
                [167158116.581, 145482676.804, 114951518.904, 88475523.2607, 65030701.9542, 46120088.6093],
                [157760706.291, 137836455.92, 113379915.298, 91641365.1536, 68924700.2428, 48698033.483],
                [143607150.179, 127875584.892, 112520771.519, 93223216.4379, 70387887.7272, 50546712.6645],
                [112483697.906, 117150813.299, 107701424.781, 97751921.982, 81366149.7358, 64815248.0104],
                [98061096.2279, 107455892.672, 104297905.683, 100185926.349, 82831840.0415, 70943170.025],
                [89137895.3342, 102529524.376, 103609158.886, 100258967.138, 82953357.8248, 74711580.7698],
                [80061011.5245, 99701974.4374, 101135778.326, 103509334.667, 93903836.9186, 82575604.803],
                [76955476.7623, 99204914.2963, 100657431.412, 105555739.515, 97037484.1735, 86439016.9643],
                [69605396.4523, 94038539.7022, 99107027.4359, 106465379.024, 99860594.6244, 91198028.5089],
                [65217286.435, 91077737.2129, 97711464.8466, 106493430.55, 102497916.497, 94414439.934],
                [56404760.6472, 89515576.0369, 96725175.6269, 110780475.382, 106003429.523, 99779805.5687],
                [57623765.1379, 86000000.115, 92645901.5832, 109963760.39, 112042014.351, 105580958.636],
                [43952333.816, 79252453.6185, 86299264.174, 117994751.33, 118933764.105, 117810670.231],
                [24634373.903, 66065001.64, 68243438.02, 135426575.793, 137419433.105, 172318869.066],
                [18589595.0428, 57728776.2495, 62320493.2527, 139634562.057, 152773746.081, 223807728.344],
                [20594648.2386, 59271804.7584, 63480011.0556, 137984419.731, 170575636.626, 249672094.732],
                [19982219.0805, 51425904.8348, 62459589.4291, 133533308.408, 173764569.114, 340432929.048],
                [14233265.2396, 48314855.5797, 69876452.1042, 118939345.179, 202726701.441, 782283022.416],
                [12050167.2655, 41971380.0582, 67231664.3917, 117242812.869, 204972074.627, 1193155462.46],
                [217614427.542, 172388819.341, 121628347.075, 83816254.3477, 55033857.4819, 34150040.9715],
                [188956032.614, 152228600.204, 119348410.647, 87194122.6816, 60562480.3003, 39268083.3223],
                [181744158.147, 148070781.167, 115670547.145, 89247318.1735, 64040594.4522, 43685819.4112],
                [151073623.188, 144854097.547, 117506093.143, 91800688.1804, 68393724.3162, 47858125.7734],
                [104973486.386, 113192497.975, 107358293.398, 99298184.4158, 82854153.6826, 66448368.5807],
                [96800968.5366, 108813789.1, 107709745.619, 100869902.616, 87949796.8523, 73853744.7042],
                [77542089.7039, 100444892.881, 104405576.351, 103448058.566, 92715073.403, 80484729.3368],
                [73902932.3516, 98357005.817, 101596272.347, 106187268.512, 94949640.7439, 84977461.0913],
                [65723772.4606, 92220226.5048, 100949356.3, 107088570.17, 98364839.1687, 88907820.7861],
                [58227272.214, 88096043.6116, 97778537.1547, 109673326.262, 103072681.201, 95546867.7228],
                [52242736.0392, 79021365.3116, 92296514.2048, 110699401.452, 111373663.316, 104556413.811],
                [42387748.6585, 75036578.343, 88900875.6779, 113953224.692, 119871722.227, 118650222.998],
                [42552145.4285, 73277455.5577, 86638982.1563, 117509435.252, 122443225.523, 127794645.949],
                [29487207.2902, 62587561.9928, 80747503.8074, 122638642.662, 133371495.651, 146929399.381],
                [232212730.923, 172748200.813, 123164310.855, 83234536.1767, 52534357.4283, 30163940.3295],
                [209046363.073, 165889951.977, 122621699.858, 83335081.7221, 54798420.2942, 34266296.8977],
                [175059723.489, 158635872.558, 119903390.149, 84477402.7274, 59083638.5898, 39711091.9335],
                [180976714.043, 154287876.699, 115600774.565, 88075679.1989, 61996037.5575, 41220965.6938],
                [166541063.013, 144991639.006, 116876672.36, 86937102.9577, 64578343.8305, 44125027.459],
                [134791895.785, 127292591.696, 113389093.855, 94065035.7771, 71569091.3325, 50277016.4088],
                [116307285.53, 120873545.356, 110371130.191, 95960676.9701, 76880992.7215, 61613992.3649],
                [93643102.1393, 106738439.582, 105458128.829, 100696412.304, 87801584.4699, 73494131.1528],
                [70155926.5125, 92617201.6151, 99897419.2984, 107979933.81, 102133656.856, 91712156.019],
                [48432684.2664, 77640488.3084, 95885230.2566, 113033574.748, 113968272.876, 105856099.732],
                [44330245.1179, 76228877.0172, 92090755.5776, 115109943.263, 116827090.805, 111978280.037],
                [41986963.3733, 76941616.7393, 91077765.8441, 114116953.114, 120770471.227, 118076659.132],
                [24350677.148, 62062462.4579, 81391127.5714, 120199164.039, 139426704.127, 139859946.552],
                [16566555.6961, 51508541.8403, 74545214.652, 131282172.01, 163307573.327, 213211499.471],
                [15667048.0325, 54516055.2198, 85268892.8266, 137304083.086, 180118662.136, 244286666.947],
                [26899712.5025, 47612160.0504, 93480391.8501, 116366130.415, 245905247.734, 859551349.959],
                [77724573.3068, 24522031.096, 59371312.1115, 85723599.4897, 636830700.998, 5457044198.07],
                [184221616.692, 157528964.72, 121670502.315, 82992698.6482, 55715276.2256, 34907386.7376],
                [186240877.897, 146467471.52, 118213111.658, 86607956.5869, 59598204.4652, 37143728.2774],
                [136253992.631, 126603539.187, 114298371.853, 91497814.6939, 72230713.8519, 53987276.2465],
                [106314561.389, 113964562.672, 107918869.145, 100428267.401, 80796162.5065, 63503161.2242],
                [47356124.0695, 77562337.7631, 96053464.4237, 113941864.156, 112215268.984, 102384975.424],
                [23750150.5736, 55415541.0036, 87983441.2884, 123408442.537, 133986521.045, 127934773.286],
                [17255606.3062, 50453390.7123, 85294570.3812, 132676718.474, 154723573.633, 163970266.721],
                [11155571.6465, 38800946.9791, 78247709.2722, 140346162.545, 224168766.021, 484901098.72],
                [200559738.269, 159078279.738, 122953889.782, 82731894.4139, 55184352.7519, 34838689.9769],
                [165171567.434, 140314635.161, 117112512.952, 86173443.7999, 63763344.3168, 43066897.3621],
                [154588410.824, 135316209.281, 116328188.041, 86979136.0898, 64298781.8186, 45675697.948],
                [162002575.451, 135860600.864, 116417372.501, 87962745.6728, 67385382.8884, 47394394.2437],
                [128859771.183, 124587199.774, 112922590.213, 94837327.0756, 77299742.6425, 59873845.1031],
                [114276325.277, 122425104.132, 113704531.172, 95586394.9436, 81568052.7778, 66492550.8975],
                [69722356.9925, 92857115.5751, 107702036.034, 104686022.445, 90162037.5295, 75155699.3395],
                [53531426.5745, 86250467.0279, 100677800.573, 111415319.959, 103181772.44, 86715237.9212],
                [35730667.9461, 66247934.2, 96896061.1902, 120020733.258, 115654081.28, 99121977.5028],
                [26298204.4428, 62679511.988, 88463462.5452, 124553583.264, 136807712.722, 111284163.958],
                [15272331.1628, 44586299.5689, 78896976.0433, 131072821.633, 154407202.907, 143787641.777],
                [10706035.275, 38633176.148, 78176970.26, 143005159.177, 184412033.8, 235682483.354],
                [9664335.59595, 36576257.9713, 75767089.7455, 138404693.488, 226474614.578, 437137873.508],
                [83175694.5772, 88438166.4869, 94345688.9479, 103217951.644, 108653050.69, 31193044.2219],
                [60303529.1302, 74640785.895, 85751272.669, 113775336.998, 126310448.118, 143318527.736],
                [127382843.883, 127391061.218, 110617916.138, 84726610.041, 75952409.0468, 75952409.0468],
                [56804333.6506, 73257905.0076, 75274303.5987, 103673126.61, 105849015.291, 113943879.481],
                [36338997.8303, 65471934.6664, 80586441.3589, 124557270.448, 133214955.509, 184503981.16],
                ]

    return dict(zip(spectypes, flux_array))




def get_object_flux(spectype=None, teff=None, lumclass='V', ):


    spec_types=['O5','O9','B0','B1','B3', 'B6', 'B8', 'A0', 'A2', 'A5',
                'F0', 'F2','F5', 'F8', 'G0', 'G2', 'G5', 'G8', 'K0', 'K2', 'K5', 'K7', 'M0',
                'M2', 'M4', 'M5']


    if spectype is None:

        if teff is None:
            print('MUST PROVIDE SPECTRAL TYPE OR TEMPERATURE.\n\n')
        if np.isnan(teff):
            teff=5800.

        teff_ranges=np.array([55_000, 38_000, 30_000, 23_000, 18_000, 15_000, 12_000, 10_000, 9_000, 8300,
                             7400, 7000, 6700, 6300, 6000, 5800, 5660, 5440, 5240, 4960, 4400, 4000, 3750, 3600, 3400, 3200])

        spectype = spec_types[np.argmin(np.abs(teff-teff_ranges))]
        flux_key = spectype+lumclass

    elif any( np.isin([spectype], ['Seyfert1', 'Seyfert2', 'QSO', 'NGC1068', 'Liner']) ):
        flux_key = spectype

    else:
        flux_key = spectype


    flux_dict = get_object_flux_dictionary()

    if not( any(np.isin([flux_key], list(flux_dict.keys()))) ):
        print(flux_key + ' NOT IN LIST OF SPECTRAL TYPES. PLEASE CHOOSE FROM BELOW: ')
        print(flux_dict.keys() )


    return flux_dict[flux_key]


def calc_n_photon_per_sec(Vmag, spectype=None,  teff=None, lumclass='V', fiber_setup='100',
                          airmass=1.5, seeing=1.0, ):

    if fiber_setup == '300':
    # arguments for Low-Res mode.
        pinhole = 2.3
        binning = 5.
        npix = 35. * 3.
        m1size = 8.4 #// effic 2019
        insttrans = np.array([1.419, 2.064, 4.454, 3.982, 6.469, 8.261])

    elif fiber_setup == '200':
        pinhole = 1.5
        binning = 2.
        npix = 17. * 5.
        m1size = 8.4 #// effic 2019
        # insttrans=[0.90,2.00,3.83,4.73,4.93,6.07];
        insttrans = [0.545, 1.697, 2.528, 3.554, 4.065, 4.045];

    elif fiber_setup == '100':
        pinhole = 0.74
        binning = 1.
        npix = 9. * 7.
        m1size = 8.4 # effic 2019
        # insttrans=[0.88,1.35,2.40,3.98,2.84,4.28];
        # effic 2021
        insttrans = [0.448, 2.015, 1.992, 1.637, 2.278, 0.986];

    elif fiber_setup=='VATT':
        pinhole = 7.5
        binning = 1.
        npix = 20. * 9.
        m1size = 1.9
        insttrans = [0.00, 0.50, 1.50, 4.00, 5.00, 4.00];
        binocular = False;

    elif fiber_setup=='POL':
        pinhole = 1.5
        binning = 2.
        npix = 17. * 5.
        m1size = 8.4
        insttrans = [0.39, 1.34, 5.10, 3.87, 7.13, 7.61];

    else:
        print('fiber_setup must be \'300\', \'200\', \'100\', \'VATT\' or \'POL\' ')


    # properties for each cross-disperser
    teltrans_arr = 0.01 * np.array(insttrans)
    flam_arr = [0.402704, 0.450081, 0.505877, 0.582962, 0.687765, 0.827176]
    fwhm_arr = np.array([0.000698, 0.000781, 0.000887, 0.000990, 0.001182, 0.001459])
    readnoise_arr = [3.5, 3.5, 3.5, 4., 4., 4.]

    # Add extinction to Vmag, calculate incoming flux
    kparr = np.array([0.3035, 0.2067, 0.1487, 0.1180, 0.0678, 0.0418])
    vmag_extincted = Vmag + kparr * airmass #extinction through atmosphere
    zeropoint_fluxarr = get_object_flux(spectype=spectype, teff=teff, lumclass=lumclass)

    # transmission through fiber pinhole
    trans = np.power(math.erf(pinhole / (seeing * 1.202)), 2)
    area = np.power(m1size, 2) * np.pi / 4.;

    n_photons_per_sec = zeropoint_fluxarr * fwhm_arr * binning * area * teltrans_arr * trans * np.power(10, -0.4 * vmag_extincted);

    return n_photons_per_sec, npix


def pepsi_snr(Vmag, exptime, spectype=None,  teff=None, lumclass='V', fiber_setup='100',
              airmass=1.5, seeing=1.0, binocular=False):


    '''
        Vmag: V-band magnitude (Vega) of your source
        exptime: Exposure time in seconds
        spectype: Spectral type of stellar source, also can be any of ['Seyfert1', 'Seyfert2', 'QSO', 'NGC1068', 'Liner']
        teff: effective temperature in Kelvin, can provide instead of Spectral Type
        resolution: 'low' (R=50,000), 'med' (R=130,000), or 'high' (R=250,000)
        binocular: True/False
        airmass: observed airmass
        seeing: observed seeing (FWHM in arcsec)
    '''

    if fiber_setup=='VATT':
        binocular=False

    n_photon_per_sec, npix = calc_n_photon_per_sec(Vmag,spectype,teff,lumclass,fiber_setup,airmass,seeing)

    # Readnoise for the Blue and Red CCD, respectively
    readnoise_arr = [3.5, 3.5, 3.5, 4., 4., 4.]

    n_photon = n_photon_per_sec * exptime
    snratio = n_photon / np.sqrt(n_photon + npix * np.power(readnoise_arr, 2.))

    if binocular:
        snratio *= np.sqrt(2.)

    return np.round(snratio, 1)



def pepsi_exptime(Vmag, snratio, spectype=None,  teff=None, lumclass='V', fiber_setup='100',
              airmass=1.5, seeing=1.0, binocular=False):


    '''
        snr: desired signal-to-noise ratio
        Vmag: V-band magnitude (Vega) of your source
        spectype: Spectral type of stellar source, also can be any of ['Seyfert1', 'Seyfert2', 'QSO', 'NGC1068', 'Liner']
        teff: effective temperature in Kelvin, can provide instead of Spectral Type
        resolution: 'low' (R=50,000), 'med' (R=130,000), or 'high' (R=250,000)
        binocular: True/False
        air: observed airmass
        see: observed seeing (FWHM in arcsec)
    '''


    n_photon_per_sec, npix = calc_n_photon_per_sec(Vmag,spectype,teff,lumclass,fiber_setup,airmass,seeing)

    if fiber_setup=='VATT':
        binocular=False

    if binocular:
        snratio /= np.sqrt(2)

    # Readnoise for the Blue and Red CCD, respectively
    readnoise_arr = [3.5, 3.5, 3.5, 4., 4., 4.]

    # Solve array equation for texp:
    # S/N = n_photon_per sec * texp / sqrt( n_photon_per_sec * texp + npix*readnoise^2 )
    a = -np.power(n_photon_per_sec, 2)
    b = np.power(snratio,2) * n_photon_per_sec
    c = np.power(snratio,2) * npix * np.power(readnoise_arr, 2.)

    exptime =  (-b - np.sqrt(b**2. - 4*a*c) ) / (2*a)

    return np.round(exptime, 1)


In [56]:
def add_targets(df: pd.DataFrame, conn: Connection=None, **kwargs) -> pd.DataFrame:
    column_prefix = "target_"
    # note that incoming df variable is ignored
    # this function intended always to be the first on in a chain
    targets = pd.read_sql(
        "select * from tom_target t;",
        #    join tom_catalogassociation ca on ca.target_id = t.id and ca.catalog = 'TESS TICv8' and ca.association = 'Primary ID'
        #    join tom_tess_ticv8 tt on tt.Identifier = ca.catalog_id
        conn,
        index_col="id",
    )
    new_column_names = {column: f"{column_prefix}{column}" for column in targets.columns}
    targets.rename(columns=new_column_names, inplace=True)
    return targets


def add_speckle(df: pd.DataFrame, conn: Connection, **kwargs) -> pd.DataFrame:
    column_prefix = "speckle_"
    count_column = f"{column_prefix}count"
    speckle = pd.read_sql(
        "select * from tom_specklerawdata;", conn, index_col="target_id"
    )
    speckle = speckle.groupby("target_id").id.agg(foo="count")
    speckle.rename(columns={"foo": count_column}, inplace=True)
    df = df.join(speckle)
    df[count_column] = df[count_column].fillna(0)
    df[f"{column_prefix}count"] = df[f"{column_prefix}count"].astype(int)
    return df


def add_spectra(df: pd.DataFrame, conn: Connection, **kwargs) -> pd.DataFrame:
    column_prefix = "pepsi_"
    count_column = f"{column_prefix}count"
    pepsi = pd.read_sql(
        "select * from tom_spectrumrawdata;", conn, index_col="target_id"
    )
    pepsi = pepsi.groupby("target_id").id.agg(foo="count")
    pepsi.rename(columns={"foo": count_column}, inplace=True)
    df = df.join(pepsi)
    df[count_column] = df[count_column].fillna(0)
    df[f"{column_prefix}count"] = df[f"{column_prefix}count"].astype(int)
    return df

def add_lists(df: pd.DataFrame, conn: Connection, **kwargs) -> pd.DataFrame:
    column_prefix = "list_"
    for (target_list,) in conn.execute("select name from tom_targetlist;").fetchall():
        list_members = [
            result[0]
            for result in conn.execute(
                """
                select t.name
                from tom_targetlist tl
                join tom_targetlist_targets tlt on tlt.targetlist_id = tl.id
                join tom_target t on t.id = tlt.target_id
                where tl.name = ?
                ;
                """,
                [target_list],
            ).fetchall()
        ]
        column_name = f"{column_prefix}{target_list.replace(" ", "_")}"
        df[column_name] = df.target_name.isin(list_members)
    return df

def add_ephemerides(df: pd.DataFrame, conn: Connection, **kwargs) -> pd.DataFrame:
    column_prefix = "ephem_"
    ephem = pd.read_sql(
        """
        select t.id, bp.member, bp.period, bp.t0_a, bp.duration_a, bp.depth_a, bp.t0_b, bp.duration_b, bp.depth_b
        from tom_binaryparameters bp
        join tom_scienceresult sr on sr.id = bp.scienceresult_ptr_id
        join tom_target t on t.id = sr.target_id
        ;""",
        conn,
        index_col="id",
    )
    new_column_names = {column: f"{column_prefix}{column}" for column in ephem.columns}
    ephem.rename(columns=new_column_names, inplace=True)
    df = df.join(ephem)
    return df

def add_tess(df: pd.DataFrame, conn: Connection, **kwargs) -> pd.DataFrame:
    column_prefix="tess_"
    tess = pd.read_sql(
        """
        select ca.target_id, tt.*
        from tom_tess_ticv8 tt
        join tom_catalogassociation ca on ca.catalog_id = tt.Identifier
        where ca.association = 'Primary ID' and ca.catalog = 'TESS TICv8'
        """,
        conn,
        index_col="target_id",
    )
    tess.drop("id", axis=1, inplace=True)
    new_column_names = {column: f"{column_prefix}{column}" for column in tess.columns}
    tess.rename(columns=new_column_names, inplace=True)
    df = df.join(tess)
    return df

def add_coords(df: pd.DataFrame, conn: Connection, **kwargs) -> pd.DataFrame:
    """Add fundamental things like coordinates from available catalog data"""
    tess_mapping = {"ra": "tess_ra", "dec": "tess_dec", "pmra": "tess_pmRA", "pmdec": "tess_pmDEC", "parallax": "tess_plx", "Vmag": "tess_Vmag", "Teff": "tess_Teff"}
    if "tess_version" in df.columns:
        for field, tess_field in tess_mapping.items():
            df[field] = df[tess_field]
    return df

def hide_cols(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    cols_to_remove = []
    if prefix := kwargs.get("prefix"):
        prefix_cols = [col for col in df.columns if col.startswith(prefix)]
        cols_to_remove += prefix_cols
    if len(cols_to_remove) > 0:
        df.drop(labels=cols_to_remove, axis=1, inplace=True)
    return df

def add_observability(df: pd.DataFrame, session_beg: Time=None, session_end: Time=None, **kwargs) -> pd.DataFrame:
    column_prefix = "observable_"
    # make astroplan objects for each target
    fixed_targets = [
        ap.FixedTarget(
            coord=SkyCoord(
                frame="icrs",
                obstime=Time("2000.0", format="jyear", scale="tdb"),
                ra=row["ra"] * u.deg,
                dec=row["dec"] * u.deg,
                pm_ra_cosdec=row["pmra"] * u.mas / u.yr,
                pm_dec=row["pmdec"] * u.mas / u.yr,
                # distance=row["parallax"] * u.pc,
            ),
            name=row["target_name"],
        )
        for (_, row) in df.iterrows()
    ]

    constraints = [
        ap.AltitudeConstraint(30 * u.deg, 80 * u.deg),
        ap.AirmassConstraint(2),
        # ap.AtNightConstraint.twilight_civil(),
    ]

    # calculate observability
    df[f"{column_prefix}observable"] = ap.is_observable(
        constraints, observer, fixed_targets, (session_beg, session_end)
    )

    # calculate rise/meridian/set times
    with warnings.catch_warnings():
        # ignore warnings thrown when targets are never or always visible
        warnings.simplefilter("ignore", AstropyWarning)
        rise_times = observer.target_rise_time(
            session_beg, fixed_targets, which="nearest"
        ).filled(session_beg)
        df[f"{column_prefix}_rise_jd"] = rise_times.jd
        df[f"{column_prefix}_rise_time"] = list(
            map(
                lambda x: x.to_datetime(timezone=observer.timezone).isoformat(
                    sep=" ", timespec="seconds"
                ),
                rise_times,
            )
        )
        meridian_times = observer.target_meridian_transit_time(
            session_beg, fixed_targets, which="nearest"
        )
        df[f"{column_prefix}meridian_jd"] = meridian_times.jd
        df[f"{column_prefix}meridian_time"] = list(
            map(
                lambda x: x.to_datetime(timezone=observer.timezone).isoformat(
                    sep=" ", timespec="seconds"
                ),
                meridian_times,
            )
        )
        set_times = observer.target_set_time(
            session_beg, fixed_targets, which="nearest"
        ).filled(session_end)
        df[f"{column_prefix}set_jd"] = set_times.jd
        df[f"{column_prefix}set_time"] = list(
            map(
                lambda x: x.to_datetime(timezone=observer.timezone).isoformat(
                    sep=" ", timespec="seconds"
                ),
                set_times,
            )
        )
        return df

# TODO: add columns for beginning, ending, and max airmass or altitude


In [81]:

def pipeline(steps=[], initial_df=None, **kwargs):
    intermediate_df = pd.DataFrame() if initial_df is None else initial_df
    for step in steps:
        intermediate_df = step(intermediate_df, **kwargs)
    return intermediate_df

targets = pipeline([add_targets,
                    add_tess,
                    add_coords,
                    hide_tess,
                    # hide_cols(prefix="tess"),
                    # add_speckle,
                    # add_observability,
                    # add_spectra,
                    add_lists,
                    # add_ephemerides,
                    ],
                   None,
                   conn=conn,
                   observer=observer,
                   session_beg=session_beg,
                   session_end=session_end,
                   )

targets[targets["list_LBT_2023B"]][["target_name","ra","dec","pmra","pmdec","Vmag"]].fillna(0).set_index("target_name")

,ra,dec,pmra,pmdec,Vmag
target_name,,,,,
TIC 219469945,241.047908,43.030301,-0.866116,-13.091300,12.507
TIC 161043618,223.425163,52.715848,2.178010,-3.594290,12.497
TIC 441794509,263.598849,74.472259,3.441650,-21.273300,12.819
TIC 367448265,78.382438,35.653053,-5.856750,-3.432870,7.900
TIC 25818450,352.743444,53.069150,3.573600,-1.172130,11.782
TIC 414026507,336.837717,56.740362,-3.664490,-2.956470,10.899
TIC 470710327,357.329052,61.962787,0.600000,-2.500000,9.650
TIC 391620600,71.640389,44.753662,-7.755290,2.230940,12.772
TIC 89278612,301.219498,32.643051,-0.485210,-1.506920,11.235


In [12]:
# TODO: following code only works for binary systems - generalize to n-member systems
joined_params = params.drop(columns=["scienceresult_ptr_id"])
joined_params = joined_params[joined_params["member"] == "A"].join(
    joined_params[joined_params["member"] == "B"],
    on="local_id",
    lsuffix="_A",
    rsuffix="_B",
)
targets = targets.join(joined_params, on="local_id", how="left")

# calculate eclipse phase timings
# TODO: following code only works for binary systems - generalize to n-member systems
# members = list(map(lambda name: name.replace("period_", "") if "period_" in name, targets.columns))
members = ["A", "B"]
components = ["a", "b"]
phase_milestones = [0.00, 0.25, 0.5, 0.75]


# some helper functions for consistent column naming
def phase_milestone_col(member, component, milestone):
    return f"{member}{component}_phase_{milestone:.2f}"


def eclipse_timing_col(member, component, event):
    return f"{member}{component}_{event}"


def entire_eclipse_col(member, component=""):
    return f"{member}{component}_entire_eclipse"


for member in members:
    targets[entire_eclipse_col(member)] = False
    for milestone in phase_milestones:
        targets[phase_milestone_col(member, "", milestone)] = False
    for component in components:
        targets[entire_eclipse_col(member, component)] = False
        targets[eclipse_timing_col(member, component, "ingress")] = False
        targets[eclipse_timing_col(member, component, "egress")] = False
        for milestone in phase_milestones:
            col_name = phase_milestone_col(member, component, milestone)
            targets[col_name] = False
            targets[f"{col_name}_jd"] = None
            targets[f"{col_name}_local"] = None
            targets[f"{col_name}_uva"] = None

for index, row in targets.iterrows():
    for member in members:
        period_col = f"period_{member}"
        if not period_col in row or math.isnan(period := row[period_col]):
            continue
        num_periods = (jd_end - jd_beg) / period
        targets.loc[index, f"num_periods_{member}"] = num_periods
        member_components = []
        for component in components:
            component_name = (
                f"t0_{component}_{member}"  # TODO: fix inconsistent column name
            )
            if not component_name in row or math.isnan(t0 := row[component_name]):
                continue
            member_components.append(component)
            t_pre = math.floor((jd_beg - t0) / period) * period + t0
            t_post = math.ceil((jd_end - t0) / period) * period + t0
            beg_phase = (jd_beg - t_pre) / period
            end_phase = 1 - (t_post - jd_end) / period
            targets.loc[index, f"beg_phase_{member}{component}"] = beg_phase
            targets.loc[index, f"end_phase_{member}{component}"] = end_phase
            duration = (
                row[f"duration_{component}_{member}"] / 24
            )  # TODO: fix inconsistent column name
            ingress = jd_beg < t_pre + period - duration < jd_end
            targets.loc[index, eclipse_timing_col(member, component, "ingress")] = (
                ingress
            )
            egress = jd_beg < t_pre + period + duration < jd_end
            targets.loc[index, eclipse_timing_col(member, component, "egress")] = egress
            targets.loc[index, entire_eclipse_col(member, component)] = ingress & egress
            # mark any phase milestones
            for milestone in phase_milestones:
                col_name = phase_milestone_col(member, component, milestone)
                if milestone == 0:
                    # phase = 0 needs to be handled differently because of the wraparound issue
                    covers_milestone = (num_periods > 1) | (end_phase < beg_phase)
                else:
                    # handle case: multiple periods guarantees milestone is hit
                    # handle case: start before milestone, end after milestone
                    # handle case: start after milestone, wrap around and end after milestone, with end < beg
                    covers_milestone = (
                        (num_periods > 1)
                        | (beg_phase < milestone < end_phase)
                        | (
                            (milestone < beg_phase)
                            & (milestone < end_phase)
                            & (end_phase < beg_phase)
                        )
                    )
                targets.loc[index, col_name] = covers_milestone
                if covers_milestone:
                    milestone_time = (
                        t_pre + period * milestone
                    )  # TODO: fix this for wraparound
                    targets.loc[index, f"{col_name}_jd"] = milestone_time
                    dt = Time(milestone_time, format="jd", scale="utc").to_datetime(
                        timezone=observer.timezone
                    )
                    targets.loc[index, f"{col_name}_time"] = dt.isoformat(
                        sep=" ", timespec="seconds"
                    )
        targets.loc[index, entire_eclipse_col(member)] = targets.loc[
            index,
            [entire_eclipse_col(member, component) for component in member_components],
        ].any()
        for milestone in phase_milestones:
            targets.loc[index, phase_milestone_col(member, "", milestone)] = (
                targets.loc[
                    index,
                    [
                        phase_milestone_col(member, component, milestone)
                        for component in member_components
                    ],
                ].any()
            )
targets["any_entire_eclipse"] = targets[
    [entire_eclipse_col(member) for member in members]
].any(axis=1)
targets["any_ingress"] = targets[
    [
        eclipse_timing_col(member, component, "ingress")
        for member in members
        for component in components
    ]
].any(axis=1)
targets["any_egress"] = targets[
    [
        eclipse_timing_col(member, component, "egress")
        for member in members
        for component in components
    ]
].any(axis=1)
for milestone in phase_milestones:
    targets[phase_milestone_col("any", "", milestone)] = targets[
        [
            phase_milestone_col(member, component, milestone)
            for member in members
            for component in components
        ]

    ].any(axis=1)
targets["pepsi fiber"] = "VATT/CDIII/CDVI"

targets

In [5]:
# mags = [(mag, count) for count, mag in enumerate(sorted(targets["Vmag"]))]
mags = list(targets["Vmag"].sort_values())

import bokeh
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.plotting import figure
from bokeh.io import show
p = figure()
p.line(list(range(len(mags))), mags)
show(p)

print (len(mags), mags)


Loading BokehJS ...

198 [6.34, 6.99, 7.09, 7.9, 7.9, 7.96, 8.815, 9.04, 9.37, 9.379, 9.526, 9.65, 9.7, 9.804, 9.94, 10.117, 10.272, 10.302, 10.306, 10.325, 10.387, 10.39, 10.403, 10.409, 10.426, 10.498, 10.524, 10.542, 10.574, 10.58, 10.581, 10.616, 10.665, 10.718, 10.734, 10.813, 10.899, 10.969, 10.975, 10.996, 10.997, 10.998, 11.002, 11.011, 11.124, 11.136, 11.157, 11.235, 11.275, 11.31, 11.33, 11.356, 11.371, 11.405, 11.429, 11.506, 11.506, 11.51, 11.568, 11.636, 11.756, 11.774, 11.782, 11.788, 11.839, 11.858, 11.899, 11.961, 11.982, 12.041, 12.049, 12.089, 12.187, 12.211, 12.262, 12.298, 12.304, 12.305, 12.309, 12.338, 12.341, 12.367, 12.367, 12.377, 12.46, 12.47, 12.497, 12.502, 12.504, 12.507, 12.539, 12.604, 12.609, 12.617, 12.617, 12.627, 12.679, 12.682, 12.712, 12.732, 12.772, 12.783, 12.802, 12.819, 12.834, 12.844, 12.88, 12.907, 12.911, 12.931, 12.942, 13.0, 13.046, 13.0524, 13.08, 13.115, 13.119, 13.194, 13.204, 13.207, 13.228, 13.242, 13.277, 13.323, 13.39, 13.45, 13.455, 13.46, 13.488, 13.51

In [6]:
# targets = (
#     pd.read_csv("WD Binaries.csv", usecols=["APOGEE_ID_1", "RA", "DEC", "Gmag", "Teff"])
#     .rename(
#         columns={"APOGEE_ID_1": "local_id", "RA": "ra", "DEC": "dec", "Gmag": "Vmag"}
#     )
# )
# targets["target_type"] = "WD Binary"
# targets["pmRA"] = 0
# targets["pmDEC"] = 0
# targets["pepsi fiber"] = "VATT/CDIII/CDVI"

# targets

In [83]:
def add_pepsi_params(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    df["pepsi_fiber"] = "300"
    df["pepsi_cd_blue"] = 3
    df["pepsi_cd_red"] = 6
    df["pepsi_snr_goal"] = 50
    df["pepsi_exposure_time"] = [
        pepsi_exptime(vmag, snr, teff=teff, fiber_setup=fiber)[cd - 1]
        for vmag, snr, teff, fiber, cd in df[
            ["Vmag", "pepsi_snr_goal", "Teff", "pepsi_fiber", "pepsi_cd_red"]
        ].values
    ]
    return df


targets = pipeline([add_pepsi_params], targets)

targets

targets[targets["list_LBT_2023B"]][["target_name","ra","dec","pmra","pmdec","Vmag", "pepsi_exposure_time"]].fillna(0).set_index("target_name")

,ra,dec,pmra,pmdec,Vmag,pepsi_exposure_time
target_name,,,,,,
TIC 219469945,241.047908,43.030301,-0.866116,-13.091300,12.507,143.1
TIC 161043618,223.425163,52.715848,2.178010,-3.594290,12.497,128.4
TIC 441794509,263.598849,74.472259,3.441650,-21.273300,12.819,190.7
TIC 367448265,78.382438,35.653053,-5.856750,-3.432870,7.900,3.7
TIC 25818450,352.743444,53.069150,3.573600,-1.172130,11.782,85.4
TIC 414026507,336.837717,56.740362,-3.664490,-2.956470,10.899,58.3
TIC 470710327,357.329052,61.962787,0.600000,-2.500000,9.650,18.4
TIC 391620600,71.640389,44.753662,-7.755290,2.230940,12.772,174.5
TIC 89278612,301.219498,32.643051,-0.485210,-1.506920,11.235,79.4


In [8]:
def add_gemini_speckle_params(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    # For more info, see https://www.gemini.edu/instrumentation/alopeke-zorro/proposal-preparation
    iq70_exposures, iq85_exposures = [], []
    for vmag in df["Vmag"].values:
        if vmag < 9:
            iq70_exposures.append(3)
            iq85_exposures.append(6)
        elif 9 <= vmag < 12:
            iq70_exposures.append(5)
            iq85_exposures.append(10)
        elif 12 <= vmag < 14:
            iq70_exposures.append(7)
            iq85_exposures.append(14)
        elif 14 <= vmag < 16:
            iq70_exposures.append(11)
            iq85_exposures.append(22)
        elif 16 <= vmag:
            iq70_exposures.append(15)
            iq85_exposures.append(30)
        elif vmag != vmag: # test for nan
            iq70_exposures.append(15)
            iq85_exposures.append(30)
        else:
            raise ValueError(f"Cannot handle Vmag of {vmag}")
    df["gemini_iq70_speckle_sequences"] = iq70_exposures
    df["gemini_iq85_speckle_sequences"] = iq85_exposures
    speckle_sequence_duration = 64 * u.s
    df["gemini_iq70_speckle_exposure"] = df["gemini_iq70_speckle_sequences"] * speckle_sequence_duration
    df["gemini_iq85_speckle_exposure"] = df["gemini_iq85_speckle_sequences"] * speckle_sequence_duration
    pointing_time = 300 * u.s
    calibaration_overhead = 1.167 # 10 minutes of calibration target per hour of science target
    photometric_overhead = 1.016 # 10 minutes of calibration per 10 hour observing session
    total_overhead = calibaration_overhead * photometric_overhead
    df["gemini_iq70_speckle_program_time"] = (df["gemini_iq70_speckle_exposure"] + pointing_time)* total_overhead
    df["gemini_iq85_speckle_program_time"] = (df["gemini_iq85_speckle_exposure"] + pointing_time)* total_overhead
    return df

targets = pipeline([add_gemini_speckle_params], targets)

targets


,target_name,target_source,target_type,ra,dec,pmra,pmdec,parallax,Vmag,Teff,...,pepsi_cd_blue,pepsi_cd_red,pepsi_snr_goal,pepsi_exposure_time,gemini_iq70_speckle_sequences,gemini_iq85_speckle_sequences,gemini_iq70_speckle_exposure,gemini_iq85_speckle_exposure,gemini_iq70_speckle_program_time,gemini_iq85_speckle_program_time
id,,,,,,,,,,,,,,,,,,,,,
3366,TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,263.430502,43.860752,-11.961600,-23.987400,1.744340,12.041,5048.88,...,3,6,50,65.3,7,14,448.0,896.0,886.882656,1418.063712
3367,TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,241.047908,43.030301,-0.866116,-13.091300,1.083280,12.507,6322.79,...,3,6,50,143.1,7,14,448.0,896.0,886.882656,1418.063712
3368,TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,227.038060,39.970240,-82.825300,60.712800,5.367930,10.580,5665.40,...,3,6,50,21.2,5,10,320.0,640.0,735.116640,1114.531680
3369,TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,69.925310,29.134584,1.092150,-1.994130,0.347484,15.116,6445.00,...,3,6,50,1582.0,11,22,704.0,1408.0,1190.414688,2025.127776
3370,TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,223.425163,52.715848,2.178010,-3.594290,2.027610,12.497,5860.00,...,3,6,50,128.4,7,14,448.0,896.0,886.882656,1418.063712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559,TIC 414969157,Kostov 2022 arXiv:2202.05790,QuadEB,141.176162,22.200757,-0.138778,-0.350210,1.274850,14.508,5358.36,...,3,6,50,747.9,11,22,704.0,1408.0,1190.414688,2025.127776
3560,TIC 27543409,Kostov 2022 arXiv:2202.05790,QuadEB,122.702004,13.567217,-3.266660,0.942613,0.518392,13.663,6421.00,...,3,6,50,415.0,7,14,448.0,896.0,886.882656,1418.063712
3561,TIC 139914081,Kostov 2023 arXiv:2309.14200,QuadEB,326.971651,-49.037832,-4.706880,-3.407270,1.214990,14.534,5229.00,...,3,6,50,723.9,11,22,704.0,1408.0,1190.414688,2025.127776


In [9]:
cutpoint = 25
criteria = {
    "Faint North":      (targets["Vmag"] > 13) & (targets["list_HQND"] == False) & (targets["dec"] > cutpoint) ,
    "Faint Equatorial": (targets["Vmag"] > 13) & (targets["list_HQND"] == False) & (-cutpoint < targets["dec"]) & (targets["dec"] < cutpoint) ,
    "Faint South":      (targets["Vmag"] > 13) & (targets["list_HQND"] == False) & (targets["dec"] < -cutpoint),
    "HQU North":        (targets["list_HQND"]) & (targets["dec"] > cutpoint),
    "HQU Equatorial":   (targets["list_HQND"]) & (-cutpoint < targets["dec"]) & (targets["dec"] < cutpoint),
    "HQU South":        (targets["list_HQND"]) & (targets["dec"] < -cutpoint),
    "Bright Southern":  (targets["Vmag"] < 13) & (targets["list_HQND"] == False) & (targets["dec"] < -cutpoint),
    "Comparison":       targets["target_name"].isin(["TIC 200094011", "TIC 348651800"]),
}

observing_lists = {name: targets[criterion] for name, criterion in criteria.items()}

for name, observing_list in observing_lists.items():
    print(f"{name:<20s} {len(observing_list):5d}")

# observing_lists

Faint North             23
Faint Equatorial        25
Faint South             34
HQU North                8
HQU Equatorial           5
HQU South                2
Bright Southern         37
Comparison               2


In [10]:
def make_gemini_speckle_list(observing_lists, filename_mapping, **kwargs):
    # csv format documented towards bottom of https://www.gemini.edu/observing/phase-i/pit/pit-description
    for name, observing_list in observing_lists.items():
        output_table = pd.DataFrame()
        output_table["Name"] = observing_list["target_name"]
        output_table["RAJ2000"] = [Angle(ra*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":") for ra in observing_list["ra"]]
        output_table["DecJ2000"] = [Angle(dec*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True) for dec in observing_list["dec"]]
        output_table["V"] = observing_list["Vmag"]
        output_table["NumSequences"] = observing_list["gemini_iq85_speckle_sequences"]
        output_table["IQ70 Program Time"] = observing_list["gemini_iq70_speckle_program_time"] / 3600
        output_table["IQ85 Program Time"] = observing_list["gemini_iq85_speckle_program_time"] / 3600
        filename = filename_mapping[name]
        if len(output_table) > 0:
            output_table.to_csv(filename, index=False)

filename_mapping = {name: f"{name.lower()}.csv" for name in observing_lists.keys()}
make_gemini_speckle_list(observing_lists, filename_mapping)


In [11]:
for seeing in ["IQ70", "IQ85"]:
    total_count, total_sequences, total_time = 0, 0, 0
    # make latex table for proposal
    print(f"Target List          & Count &   Sequences   &    {seeing} \\\\")
    for name, observing_list in observing_lists.items():
        num_sequences = np.sum(observing_list[f"gemini_{seeing.lower()}_speckle_sequences"])
        program_time = np.sum(observing_list[f"gemini_{seeing.lower()}_speckle_program_time"])
        if num_sequences > 0:
            total_count += len(observing_list)
            total_sequences += num_sequences
            total_time += program_time
            print(f"{name:<20s} & {len(observing_list):4d}  &      {num_sequences:4d}     &  {program_time/3600:6.2f} \\\\")
    if total_count > 0:
        print(f"{'TOTAL':<20s} & {total_count:4d}  &      {total_sequences:4d}     &  {total_time/3600:6.2f} \\\\")
    print()


Target List          & Count &   Sequences   &    IQ70 \\
Faint North          &   23  &       213     &    6.76 \\
Faint Equatorial     &   25  &       231     &    7.34 \\
Faint South          &   34  &       334     &   10.40 \\
HQU North            &    8  &        48     &    1.80 \\
HQU Equatorial       &    5  &        27     &    1.06 \\
HQU South            &    2  &        12     &    0.45 \\
Bright Southern      &   37  &       207     &    8.02 \\
Comparison           &    2  &        10     &    0.41 \\
TOTAL                &  136  &      1082     &   36.24 \\

Target List          & Count &   Sequences   &    IQ85 \\
Faint North          &   23  &       426     &   11.25 \\
Faint Equatorial     &   25  &       462     &   12.21 \\
Faint South          &   34  &       668     &   17.44 \\
HQU North            &    8  &        96     &    2.81 \\
HQU Equatorial       &    5  &        54     &    1.63 \\
HQU South            &    2  &        24     &    0.70 \\
Bright Southe

In [13]:
cols_basic = ["local_id", "ra", "dec", "Vmag"]
cols_rawdata = ["Num Speckle", "Num Spectra"]
cols_ephem = ["period_A", "period_B"]
cols_eclipse = [f"{member}_entire_eclipse" for member in members]
cols_rv = [f"{member}{component}_phase_{milestone}" for member in members for component in ["a", "b"] for milestone in [0.25, 0.75]]
cols_common = cols_basic + cols_rawdata + cols_ephem


NameError: name 'members' is not defined

: 

In [ ]:
# min_per, max_per = 1.5, 5
# criteria={
#     # "test": ((targets["HQND"]), ""),
#     "Other Bright with spectra and speckle": ((targets["magnitude"] < 11) & (targets["Num Speckle"] > 0) & (targets["Num Spectra"] > 0) & (targets["HQND"] == False), "Low"),
#     "HQND, Dim": ((targets["HQND"]) & (targets["magnitude"]>13), "Low"),
#     "HQND Bright no spectra": ((targets["HQND"]) & (targets["Num Spectra"] == 0) & (targets["magnitude"] < 13), "Medium"),
#     "HQND Bright with spectra": ((targets["HQND"]) & (targets["Num Spectra"] > 0) & (targets["magnitude"] < 13), "High"),
#     "Featured": ((targets["Featured targets"]), "Highest"),
    # "Bright": (targets["GAIAmag"] < 19, "Low"),
    # "Light curve": (targets["any_entire_eclipse"], "Medium"),
    # "Between eclipses": ((targets["phases_A"] > 1) | (targets["phases_B"] > 1), "High"),
    # "VATT Test": ((targets["period_A"] > max_per) & (targets["period_B"] < min_per) & (targets["period_B"] > 0)
    #     | (targets["period_A"] < min_per) & (targets["period_B"] > max_per) & (targets["period_A"] > 0), "Highest"),
# }

criteria = {
    "Best RV": ((targets["any_phase_0.25"] | targets["any_phase_0.75"])
                & ((targets["Num Speckle"] > 0) | (targets["Num Spectra"] > 0))
                & (targets["Vmag"] < 12), "Highest"),
    # "WD Binary": (targets["target_type"] == "WD Binary", "Highest"),
}

matching_ids = {}
combined_ids = set()
targets["Priority"] = ""
targets["Criterion"] = ""
for name, (criterion, priority) in criteria.items():
    filtered_ids = set(targets[criterion  & (targets["Observable"])]["local_id"])
    targets.loc[targets.local_id.isin(filtered_ids), "Priority"] = priority
    targets.loc[targets.local_id.isin(filtered_ids), "Criterion"] = name
    print(f"{len(filtered_ids):3d} targets from criterion: {name}")
    combined_ids=combined_ids | filtered_ids
    matching_ids[name] = filtered_ids
print(f"Total of {len(combined_ids)} targets")

observing_list = targets[targets["local_id"].isin(combined_ids)]
# observing_list[cols_common + cols_rv + ["Criterion", "PEPSI exp time"]].sort_values("Vmag")
observing_list.sort_values("Vmag")

C:\Users\User\AppData\Local\Temp\ipykernel_51368\690768490.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  targets["Priority"] = ""
C:\Users\User\AppData\Local\Temp\ipykernel_51368\690768490.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  targets["Criterion"] = ""


 14 targets from criterion: Best RV
Total of 14 targets


,id,local_id,source,target_type,id,catalog,catalog_id,association,id,version,...,PEPSI exp time,Observable,Rise Time JD,Rise Time,Meridian Time JD,Meridian Time,Set Time JD,Set Time,Priority,Criterion
target_id,,,,,,,,,,,,,,,,,,,,,
3375,3375,TIC 367448265,Kostov 2022 arXiv:2202.05790,QuadEB,1033,TESS TICv8,367448265,Primary ID,224,20190415,...,339.1,True,2.460361e+06,2024-02-20 11:43:58-07:00,2.460362e+06,2024-02-20 19:32:29-07:00,2.460362e+06,2024-02-21 03:21:06-07:00,Highest,Best RV
3398,3398,TIC 344541836,Kostov 2022 arXiv:2202.05790,QuadEB,1056,TESS TICv8,344541836,Primary ID,290,20190415,...,156.8,True,2.460362e+06,2024-02-20 18:34:04-07:00,2.460361e+06,2024-02-20 11:30:43-07:00,2.460362e+06,2024-02-21 06:31:44-07:00,Highest,Best RV
3385,3385,TIC 470710327,Kostov 2022 arXiv:2202.05790,QuadEB,1043,TESS TICv8,470710327,Primary ID,247,20190415,...,1699.7,True,2.460362e+06,2024-02-20 18:34:04-07:00,2.460361e+06,2024-02-20 14:08:43-07:00,2.460362e+06,2024-02-21 06:31:44-07:00,Highest,Best RV
3434,3434,TIC 200094011,Kostov 2022 arXiv:2202.05790,QuadEB,1092,TESS TICv8,200094011,Primary ID,261,20190415,...,822.7,True,2.460361e+06,2024-02-20 14:05:52-07:00,2.460362e+06,2024-02-20 20:05:40-07:00,2.460362e+06,2024-02-21 02:05:28-07:00,Highest,Best RV
3430,3430,TIC 317863971,Kostov 2022 arXiv:2202.05790,QuadEB,1088,TESS TICv8,317863971,Primary ID,379,20190415,...,2837.8,True,2.460361e+06,2024-02-20 15:33:54-07:00,2.460362e+06,2024-02-20 21:40:35-07:00,2.460362e+06,2024-02-21 03:47:15-07:00,Highest,Best RV
3472,3472,TIC 286470992,Kostov 2022 arXiv:2202.05790,QuadEB,1130,TESS TICv8,286470992,Primary ID,265,20190415,...,3165.0,True,2.460362e+06,2024-02-20 18:34:04-07:00,2.460362e+06,2024-02-20 17:20:57-07:00,2.460362e+06,2024-02-21 06:31:44-07:00,Highest,Best RV
3477,3477,TIC 392229331,Kostov 2022 arXiv:2202.05790,QuadEB,1135,TESS TICv8,392229331,Primary ID,350,20190415,...,3775.5,True,2.460362e+06,2024-02-20 18:34:04-07:00,2.460362e+06,2024-02-20 17:58:44-07:00,2.460362e+06,2024-02-21 06:31:44-07:00,Highest,Best RV
3396,3396,TIC 322727163,Kostov 2022 arXiv:2202.05790,QuadEB,1054,TESS TICv8,322727163,Primary ID,204,20190415,...,5362.6,True,2.460362e+06,2024-02-21 01:30:50-07:00,2.460361e+06,2024-02-20 10:58:21-07:00,2.460362e+06,2024-02-20 20:21:57-07:00,Highest,Best RV
3392,3392,TIC 123098844,Kostov 2022 arXiv:2202.05790,QuadEB,1050,TESS TICv8,123098844,Primary ID,367,20190415,...,4123.6,True,2.460362e+06,2024-02-21 00:17:40-07:00,2.460361e+06,2024-02-20 08:58:06-07:00,2.460362e+06,2024-02-20 17:34:35-07:00,Highest,Best RV


In [ ]:
#print out results in LBT readme format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("TargetName           RA (J2000)     DEC (J2000)       Gmag      pmRA    pmDEC     Mode               ExposureTime   ExecutionTime         Priority  Notes")
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = target["PEPSI exp time"]
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f'{target["local_id"]:<19s} '
        f'{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    '
        f'{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      '
        f'{target["Vmag"]:5.2f}    '
        f'{target["pmRA"]:6.2f}   '
        f'{target["pmDEC"]:6.2f}     '
        f'{target["pepsi fiber"]:<17s}  '
        f'1x {exposure:5.0f} sec  '
        f'{readout_sec:6.0f} sec = {readout_min:5.1f}min  '
        f'{target["Priority"]:<7s}   '
        f'A & B Periods: {target["period_A"]:5.2f} & {target["period_B"]:5.2f}'
        # '--'
    )

TargetName           RA (J2000)     DEC (J2000)       Gmag      pmRA    pmDEC     Mode               ExposureTime   ExecutionTime         Priority  Notes
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
TIC 283940788        0:35:24.34    +62:54:05.74      11.77     -2.73    -0.15     VATT/CDIII/CDVI    1x  8554 sec    8694 sec = 144.9min  Highest   A & B Periods:  0.88 &  8.17
TIC 328181241        2:52:36.95     +3:20:52.38      11.64     -4.67    -9.87     VATT/CDIII/CDVI    1x  4625 sec    4765 sec =  79.4min  Highest   A & B Periods: 22.63 & 26.42
TIC 286470992        3:01:19.37    +60:34:20.26      10.32      0.15    -0.17     VATT/CDIII/CDVI    1x  3165 sec    3305 sec =  55.1min  Highest   A & B Periods:  3.11 &  4.13
TIC 392229331        3:39:04.30    +61:03:51.13      10.62      1.14     1.35     VATT/CDIII/CDVI    1x  3776 sec    3916 sec =  

In [ ]:
#output targets in LBT obs.txt format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("#TargetName         RA(J2000)      DEC(J2000)        VMag      MODE               t_exp")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = target["PEPSI exp time"]
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f'{target["local_id"]:<15s}  '
        f'{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    '
        f'{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      '
        f'{target["Vmag"]:5.2f}     '
        f'{target["pepsi fiber"]:<18s} '
        f'1x {exposure:4.0f} sec   '
    )

#TargetName         RA(J2000)      DEC(J2000)        VMag      MODE               t_exp
TIC 283940788     0:35:24.34    +62:54:05.74      11.77     VATT/CDIII/CDVI    1x 8554 sec   
TIC 328181241     2:52:36.95     +3:20:52.38      11.64     VATT/CDIII/CDVI    1x 4625 sec   
TIC 286470992     3:01:19.37    +60:34:20.26      10.32     VATT/CDIII/CDVI    1x 3165 sec   
TIC 392229331     3:39:04.30    +61:03:51.13      10.62     VATT/CDIII/CDVI    1x 3776 sec   
TIC 139650665     4:22:24.59    -18:54:58.98      11.43     VATT/CDIII/CDVI    1x 4044 sec   
TIC 367448265     5:13:31.79    +35:39:10.99       7.90     VATT/CDIII/CDVI    1x  339 sec   
TIC 200094011     5:47:07.91     +0:17:56.19       9.70     VATT/CDIII/CDVI    1x  823 sec   
TIC 239872462     5:50:03.40    +34:25:03.34      11.33     VATT/CDIII/CDVI    1x 3692 sec   
TIC 336882813     6:09:33.75    +14:37:44.35      11.79     VATT/CDIII/CDVI    1x 6498 sec   
TIC 317863971     7:22:16.20     +3:01:54.93      10.31     VATT/C

In [ ]:
asd = targets[targets.local_id == "TIC 283940788"]["Rise Time JD"].values
asd

array([2460361.56533159])

In [ ]:
task_list = []
for _, row in observing_list.iterrows():
    for member in members:
        for component in ["a", "b"]:
            for milestone in [0.25, 0.75]:
                col_name = phase_milestone_col(member, component, milestone)
                if row[col_name]:
                    if  row["Rise Time JD"] < row[f"{col_name}_jd"] < row["Set Time JD"]:
                        period = row[f"period_{member}"]
                        task_list.append(
                            (
                                row["local_id"],
                                Angle(row["ra"] * u.deg).to_string(
                                    unit=u.hour, decimal=False, precision=2, sep=":"
                                ),
                                Angle(row["dec"] * u.deg).to_string(
                                    unit=u.deg,
                                    decimal=False,
                                    precision=2,
                                    sep=":",
                                    alwayssign=True,
                                ),
                                f"{member}{component}",
                                milestone,
                                period,
                                row[col_name + "_time"],
                                row["Vmag"],
                                row["Teff"],
                                row["PEPSI exp time"] / 60,
                            )
                        )

observing_tasks = pd.DataFrame(
    task_list,
    columns=[
        "Target ID",
        "RA",
        "Dec",
        "Component",
        "Phase",
        "Period (d)",
        "LBT Time",
        "Vmag",
        "Teff",
        "Exp Time (min)",
    ],
)
observing_tasks.sort_values("LBT Time")#.to_excel("Phase timing.xlsx")

,Target ID,RA,Dec,Component,Phase,Period (d),LBT Time,Vmag,Teff,Exp Time (min)
0,TIC 367448265,5:13:31.79,+35:39:10.99,Aa,0.75,0.418238,2024-02-20 18:52:06-07:00,7.900,9212.00,5.651667
2,TIC 344541836,21:11:24.17,+57:37:13.48,Aa,0.75,2.409932,2024-02-20 20:12:49-07:00,7.900,0.00,2.613333
4,TIC 344541836,21:11:24.17,+57:37:13.48,Ba,0.25,2.755276,2024-02-20 21:18:43-07:00,7.900,0.00,2.613333
5,TIC 344541836,21:11:24.17,+57:37:13.48,Bb,0.75,2.755276,2024-02-20 21:22:41-07:00,7.900,0.00,2.613333
18,TIC 139650665,4:22:24.59,-18:54:58.98,Ab,0.25,2.091887,2024-02-20 21:50:36-07:00,11.429,5515.00,67.401667
17,TIC 139650665,4:22:24.59,-18:54:58.98,Aa,0.75,2.091887,2024-02-20 21:53:00-07:00,11.429,5515.00,67.401667
14,TIC 286470992,3:01:19.37,+60:34:20.26,Ba,0.75,4.128543,2024-02-20 22:06:55-07:00,10.325,8693.00,52.750000
1,TIC 470710327,23:49:18.97,+61:57:46.03,Aa,0.25,1.104686,2024-02-20 22:08:30-07:00,9.650,8986.00,28.328333
8,TIC 317863971,7:22:16.20,+3:01:54.93,Ba,0.75,3.733625,2024-02-20 22:11:25-07:00,10.306,8506.00,47.296667
11,TIC 336882813,6:09:33.75,+14:37:44.35,Bb,0.75,6.422862,2024-02-20 22:23:37-07:00,11.788,6121.22,108.295000


In [ ]:
from astroquery.vizier import Vizier

Vizier(catalog="II/246").query_object("2M10271222+1119016")

Empty TableList

In [ ]:
from astroquery.ipac.irsa import Irsa

Irsa.query_tap(query="select * from fp_psc where id = '2M10271222+1119016'")


AttributeError: 'IrsaClass' object has no attribute 'query_tap'